In [1]:
!pip install pymorphy2
!pip install vk_api

In [1]:
import pandas as pd
import nltk
nltk.download('punkt')
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ttais\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import re
import vk_api
import random
import datetime
import time

In [4]:
def author_and_date_creation_extraction(authorship_string):

     # паттерны для извлечения из строки с авторством
        # информации о дате создания (если указана), авторах и вдохновителях
    # Пример строки с авторством: [id36264884|B0JI] muzz [id184439531|∀кын] & [id31852381|Phomiczoff], 2015
    creation_date_detection = ', (?P<date>[0-9][0-9][0-9][0-9])'
    muzz_splitter = '( muzz )|( музз )|( муzzz )|( с реверансом )|( посв\.? )'
    user_from_id_detection = r'\[(?P<user_id>((id)|(club))[0-9]+)\|(?P<user_name>[^\]]+)\]'
    # для обработки случаев, когда ссылка на автора дана в следующем формате: [https://vk.com/olessius|oless]
    user_from_link_detection = r'\[(?P<user_id>https:\/\/vk\.com\/[A-Za-z\._]+)\|(?P<user_name>[^\]]+)\]'
    
    # извлечения даты создания
    match = re.search(creation_date_detection, authorship_string)
    if match != None:
        creation_date = int(match.group('date'))
    else:
        creation_date = None
        
    # разделение строки на часть с указанием авторства и часть с указанием вдохновителей (если есть)
    match = re.search(muzz_splitter, authorship_string)
    if match != None:
        start = match.span()[0]
        end = match.span()[1]
        authors = authorship_string[:start]
        #TODO в дальнейшем можно лемматизировать вдохновителей (muzz), если они указаны через "посв."
        muzz_string = authorship_string[end:]
    else:
        authors = authorship_string
        muzz_string = None

    # извлечение псевдонимов авторов и id профилей (или ссылки на профиль)
    if re.search(user_from_id_detection, authors) != None:
        match_iter = re.finditer(user_from_id_detection, authors)
    else:
        match_iter = re.finditer(user_from_link_detection, authors)
    author_list = []
    for match in match_iter:
        author_id_or_link = match.group('user_id')
        author_name = match.group('user_name')
        author_list.append([author_id_or_link, author_name])

    return [author_list, creation_date, muzz_string]

In [ ]:
with open('token.txt', encoding='utf-8') as file:
    token = file.read()

In [ ]:
token = input("Введите свой токен: ")

In [7]:
posts_id = []
texts = []
dates = []
authors_all = []
creation_dates = []
for offset in range(100, 1001, 100):
    vk = vk_api.VkApi(token=token)
    vk_session = vk.get_api()
    poroshok = vk_session.wall.get(owner_id = -31481258, offset = offset, count = 100)
    for post in poroshok['items']:
        post_id = int(post['id'])
        date = datetime.datetime.fromtimestamp(int(post['date'])).strftime('%Y-%m-%d %H:%M:%S')
        text_author_list = post['text'].split('©')
        text = text_author_list[0]
        # TODO в дальнейшем можно добавить в базу данных вдохновителей (muzz) и тоже атрибутировать их к постам
        authors, creation_date, muzz_string = author_and_date_creation_extraction(text_author_list[1])
        posts_id.append(post_id)
        texts.append(text)
        dates.append(date)
        authors_all.append(authors)
        if creation_date is None:
          creation_dates.append("Unknown")
        else:
          creation_dates.append(int(creation_date))

    print(f"Finished first {offset} posts")
    time.sleep(random.uniform(2, 10))

Finished first 100 posts
Finished first 200 posts
Finished first 300 posts
Finished first 400 posts
Finished first 500 posts
Finished first 600 posts
Finished first 700 posts
Finished first 800 posts
Finished first 900 posts
Finished first 1000 posts


In [25]:
df = pd.DataFrame({'text_id': posts_id, 'text': texts, 'datetime': dates,
                   'authors': authors_all, 'year_of_creation': creation_dates})

In [26]:
morph = MorphAnalyzer()

In [27]:
tags = []
lemmas = []
text_id = []
tokens = []
number_in_t = []
authors_id = []
authors_names = []
auth_nm_texts = []
for ind in df.index:
  text_tok = word_tokenize(df['text'][ind])
  i = 0
  auth_nm = []
  for token in text_tok:
    number_in_t.append(i)
    i += 1
    parsed = morph.parse(token)[0]
    tags.append(parsed.tag.POS)
    lemmas.append(parsed.normal_form)
    text_id.append(df['text_id'][ind])
    tokens.append(token)
  for item in df['authors'][ind]:
    authors_id.append(item[0])
    authors_names.append(item[1])
    auth_nm.append(item[1])
  auth_nm_texts.append(", ".join(auth_nm))

df_new = pd.DataFrame({'text_id': text_id, 'word': tokens, 'lemma': lemmas, 'tag': tags})
authors_metadata = pd.DataFrame({'author_id': authors_id, 'author_name': authors_names})
authors_metadata = authors_metadata.drop_duplicates(subset=['author_id'])

In [41]:
authors_metadata.to_csv('author_id.csv')

In [28]:
df['authors'] = auth_nm_texts
df

,text_id,text,datetime,authors,year_of_creation
0,107070,узнал я дочка что драконы \nлишь девственниц п...,2023-07-17 12:00:00,тодибо,Unknown
1,107061,олег орёт на весь царьгород\nну кто теперь тут...,2023-07-16 12:00:00,FeelGood,Unknown
2,107060,у аристарха в манной каше\nлежал такой огромны...,2023-07-15 12:00:00,зажатый,2017
3,107059,олега очень возмущает\nсуществование глистов\n...,2023-07-14 12:00:00,"тры, Мурка",Unknown
4,107058,критяне смотрятся отсюда\nкак насекомые на мхе...,2023-07-13 12:00:00,Нестер Пим,Unknown
...,...,...,...,...,...
995,104088,при виде самки уебубля\nхудожник уголь взял и ...,2021-09-20 09:37:03,Андрей Валерьевич,Unknown
996,104067,аркадий топает за деньги\nи не щадя голеностоп...,2021-09-17 06:23:31,журкова,Unknown
997,104057,мадам я весь пылаю страстью\nя не мадам я мадм...,2021-09-16 10:21:57,Андрей Валерьевич,Unknown
998,104045,неслась офелия к обрыву\nпытаясь милого забыть...,2021-09-13 08:44:52,"света-конфета, metafola",Unknown


In [29]:
df.to_csv('text_id.csv')

In [30]:
word_id = pd.DataFrame({'word': list(set(tokens))})
word_id = word_id.reset_index()
word_id = word_id.rename(columns={"index": "id"})
word_id.to_csv('word_id.csv')

In [31]:
lemma_id = pd.DataFrame({'lemma': list(set(lemmas))})
lemma_id = lemma_id.reset_index()
lemma_id = lemma_id.rename(columns={"index": "id"})
lemma_id.to_csv('lemma_id.csv')

In [32]:
words = []
lemmas = []
for word in word_id.word:
  words.append(word)
  lemmas.append(morph.parse(word)[0].normal_form)
word_to_lemma = pd.DataFrame({'word': words, 'lemma': lemmas})
word_to_lemma.to_csv('word_to_lemma.csv')

In [33]:
word_to_lemma = word_to_lemma.merge(word_id, how='left', on='word')

In [34]:
word_to_lemma = word_to_lemma.rename(columns={"id": "word_id"})
word_to_lemma = word_to_lemma.drop(columns=['word'])

In [35]:
word_to_lemma = word_to_lemma.merge(lemma_id, how='left', on='lemma')

In [36]:
word_to_lemma = word_to_lemma.rename(columns={"id": "lemma_id"})
word_to_lemma = word_to_lemma.drop(columns=['lemma'])

In [37]:
word_to_lemma.to_csv('word_to_lemma.csv')

In [38]:
df_new_merged = df_new.merge(word_id, how='left', on='word')

In [39]:
df_new_merged = df_new_merged.rename(columns={"id": "word_id"})
df_new_merged = df_new_merged.drop(columns=['word'])

In [40]:
df_new_merged['text_pos'] = number_in_t
df_new_merged.to_csv('word_data.csv')